In [4]:
import numpy as np
from dask import do
import dask.array as da
import dask.multiprocessing
from pycalphad import Database, calculate
db_alfe = Database('alfe_sei.TDB')
my_phases_alfe = ['LIQUID', 'HCP_A3', 'AL5FE2', 'AL2FE', 'AL5FE4', 'FCC_A1', 'B2_BCC', 'AL13FE4']

In [7]:
def da_calc(db_alfe, my_phases_alfe, temp, press):
    calculate(db_alfe, ['AL', 'FE', 'VA'], my_phases_alfe, mode='numpy', output='GM', T=temp, P=press)
    return np.eye(1)

dsk = {}
temp = [300] * 2
press = [1e5] * 2
num_tasks_x = len(temp)
num_tasks_y = len(press)

for idx_x in range(num_tasks_x):
    for idx_y in range(num_tasks_y):
        dsk[('x', idx_x, idx_y)] = (da_calc, db_alfe, my_phases_alfe, temp[idx_x], press[idx_y])

chunks = (tuple(np.ones(num_tasks_x)), tuple(np.ones(num_tasks_y)))
chunks_big = (tuple([5] * (num_tasks_x//5)), tuple([5] * (num_tasks_y//5)))

x = da.Array(dsk, 'x', chunks)
#%timeit x.compute(get=dask.async.get_sync)
#%timeit x.compute(get=dask.threaded.get)
#y = da.Array(dsk, 'x', chunks_big)
%time x.compute(get=dask.async.get_sync)
%time x.compute(get=dask.threaded.get)

CPU times: user 4.19 s, sys: 0 ns, total: 4.19 s
Wall time: 4.2 s


RuntimeError: cannot release un-acquired lock

Traceback
---------
  File "/home/rotis/anaconda/envs/calphadpy3/lib/python3.5/site-packages/dask/async.py", line 264, in execute_task
    result = _execute_task(task, data)
  File "/home/rotis/anaconda/envs/calphadpy3/lib/python3.5/site-packages/dask/async.py", line 246, in _execute_task
    return func(*args2)
  File "<ipython-input-7-ba78c0a4710c>", line 2, in da_calc
    calculate(db_alfe, ['AL', 'FE', 'VA'], my_phases_alfe, mode='numpy', output='GM', T=temp, P=press)
  File "/home/rotis/git/pycalphad/pycalphad/core/calculate.py", line 216, in calculate
    out = getattr(mod, output)
  File "/home/rotis/git/pycalphad/pycalphad/model.py", line 117, in <lambda>
    energy = GM = property(lambda self: self.ast)
  File "/home/rotis/git/pycalphad/pycalphad/model.py", line 108, in ast
    return Add(*list(self.models.values()))


In [6]:
temp = [300, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100] * 5
press = [1e5, 2e5, 3e5, 4e5, 5e5] * 5
%time calculate(db_alfe, ['AL', 'FE', 'VA'], my_phases_alfe, mode='numpy', output='GM', T=temp, P=press)

CPU times: user 16 s, sys: 5.92 s, total: 22 s
Wall time: 21.9 s


<xarray.Dataset>
Dimensions:       (P: 25, T: 50, component: 2, internal_dof: 5, points: 16018)
Coordinates:
  * P             (P) float64 1e+05 2e+05 3e+05 4e+05 5e+05 1e+05 2e+05 ...
  * component     (component) <U2 'AL' 'FE'
  * T             (T) float64 300.0 1.3e+03 1.4e+03 1.5e+03 1.6e+03 1.7e+03 ...
  * internal_dof  (internal_dof) int64 0 1 2 3 4
  * points        (points) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
Data variables:
    Phase         (P, T, points) <U7 'AL13FE4' 'AL13FE4' 'AL13FE4' 'AL13FE4' ...
    GM            (P, T, points) float64 -3.543e+04 -3.638e+04 -3.586e+04 ...
    Y             (P, T, points, internal_dof) float64 1.0 1.0 1.0 0.0 nan ...
    X             (P, T, points, component) float64 0.765 0.235 0.7275 ...